In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import argparse
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Reshape,Lambda
from keras.models import Model
from keras.datasets import mnist
from keras.losses import mse, binary_crossentropy
from keras.utils import plot_model
from keras import backend as K
from keras.optimizers import adam

Using TensorFlow backend.
/Users/felixgeorge/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/felixgeorge/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/felixgeorge/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/felixgeorge/opt/ana

In [2]:
def prepare_data():
    dat=[]
    frames=os.listdir('data')
    for i in frames:
        try:
            temp=cv2.imread('data/'+i)
            t=cv2.resize(temp,(64,64))
            dat.append(t)
        except Exception as e:
            continue
    return dat

In [3]:
data=prepare_data()
dat=[]
for i in data:
    if np.array(i).shape==(64,64,3):
        dat.append(np.array(i/255))
dat=np.array(dat)
dat.shape

(200, 64, 64, 3)

In [4]:
def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

In [5]:
input_img=Input(shape=(64,64,3))
x=Conv2D(8,kernel_size=3,activation='relu',padding='same')(input_img)
x=MaxPooling2D((2,2),padding='same')(x)
x=Conv2D(4,kernel_size=3,activation='relu',padding='same')(x)
x=MaxPooling2D((2,2),padding='same')(x)
x=Conv2D(2,kernel_size=3,activation='relu',padding='same')(x)
x=MaxPooling2D((2,2),padding='same')(x)
x=Flatten()(x)
x=Dense(16,activation='relu')(x)
z_mean=Dense(2,name='mean')(x)
z_var=Dense(2,name='var')(x)

z=Lambda(sampling,output_shape=(2,),name='z')([z_mean,z_var])
encoder=Model(input_img,[z_mean,z_var,z],name='encoder')
encoder.summary()

latent_inp=Input(shape=(2,),name='z_sampled')
x=Dense(16,activation='relu')(latent_inp)
x=Dense(128,activation='relu')(x)
x=Reshape((8,8,2))(x)
x=UpSampling2D((2,2))(x)
x=Conv2D(2,kernel_size=3,activation='relu',padding='same')(x)
x=UpSampling2D((2,2))(x)
x=Conv2D(2,kernel_size=3,activation='relu',padding='same')(x)
x=UpSampling2D((2,2))(x)
x=Conv2D(3,kernel_size=3,activation='sigmoid',padding='same')(x)
decoder=Model(latent_inp,output=x)
decoder.summary()


Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 64, 64, 8)    224         input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 32, 32, 8)    0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 32, 32, 4)    292         max_pooling2d_1[0][0]            
___________________________________________________________________________________________

/Users/felixgeorge/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: UserWarning: Update your `Model` call to the Keras 2 API: `Model(Tensor("z_..., outputs=Tensor("co...)`


In [6]:
from keras.losses import mse
image_size=64
original_dim = image_size * image_size

def vae_loss(x, x_decoded_mean):
    mse_loss = K.mean(mse(x, x_decoded_mean), axis=(1,2)) * image_size * image_size
    kl_loss = - 0.5 * K.mean(1 + z_var - K.square(z_mean) - K.exp(z_var), axis=-1)
    return mse_loss + kl_loss

In [7]:
output=decoder(encoder(input_img)[2])
vae = Model(input_img, output, name='vae_mlp')
vae.compile(loss=vae_loss,optimizer='adam')

In [8]:
vae.fit(dat,dat,epochs=20)


Epoch 1/20
200/200 [==============================] - 6s 28ms/step - loss: 384.9126
Epoch 2/20
200/200 [==============================] - 2s 8ms/step - loss: 381.9935
Epoch 3/20
200/200 [==============================] - 1s 6ms/step - loss: 379.1840
Epoch 4/20
200/200 [==============================] - 1s 6ms/step - loss: 376.1680
Epoch 5/20
200/200 [==============================] - 1s 5ms/step - loss: 372.8130
Epoch 6/20
200/200 [==============================] - 1s 5ms/step - loss: 368.7194
Epoch 7/20
200/200 [==============================] - 1s 5ms/step - loss: 363.7104
Epoch 8/20
200/200 [==============================] - 1s 5ms/step - loss: 356.2279
Epoch 9/20
200/200 [==============================] - 1s 5ms/step - loss: 345.7983
Epoch 10/20
200/200 [==============================] - 1s 5ms/step - loss: 330.8311
Epoch 11/20
200/200 [==============================] - 1s 5ms/step - loss: 313.0445
Epoch 12/20
200/200 [==============================] - 1s 5ms/step - loss: 293.2288

In [10]:
a=encoder.predict(dat[:2])

In [11]:
b=decoder.predict(a[2])

In [12]:
b.shape

(2, 64, 64, 3)

In [ ]:
plt.imshow(b[0])

In [ ]:
plt.plot([i for i in range(10)])